In [3]:
import requests
from pyvis.network import Network
import webbrowser
from pathlib import Path

def fetch_bitcoin_block_data(block_height):
    """Fetch transaction data for a specific block height."""
    url = f"http://185.239.209.254:9900/v1/funds-flow/bitcoin/get_block"
    params = {
        "block_height": block_height,
        "response_type": "graph"
    }
    headers = {
        "accept": "application/json"
    }
    
    response = requests.get(url, params=params, headers=headers)
    return response.json()

def create_transaction_graph(data, output_path="bitcoin_transactions.html"):
    """Create and visualize a graph of Bitcoin transactions using Pyvis."""
    # Initialize the network
    net = Network(
        height="750px", 
        width="100%", 
        bgcolor="#ffffff", 
        font_color="black",
        notebook=True
    )
    
    # Configure physics layout
    net.set_options("""
    {
        "physics": {
            "forceAtlas2Based": {
                "gravitationalConstant": -100,
                "centralGravity": 0.01,
                "springLength": 100,
                "springConstant": 0.08
            },
            "maxVelocity": 50,
            "solver": "forceAtlas2Based",
            "timestep": 0.35
        }
    }
    """)
    
    # Process nodes and edges
    for item in data:
        if item["type"] == "node":
            node_id = item["id"]
            
            # Set node properties based on type
            if item["label"] == "transaction":
                title = f"""
                Transaction Details:
                Amount: {item.get('amount')} BTC
                Block Height: {item.get('block_height')}
                Timestamp: {item.get('timestamp')}
                ID: {node_id}
                """
                color = "#ff7f0e"  # Orange for transactions
                shape = "box"
                size = 30
            else:  # Address nodes
                title = f"""
                Address: {item.get('address')}
                ID: {node_id}
                """
                color = "#1f77b4"  # Blue for addresses
                shape = "dot"
                size = 25
            
            # Add node to network
            net.add_node(
                node_id,
                label=item["label"],
                title=title,
                color=color,
                shape=shape,
                size=size
            )
            
        elif item["type"] == "edge":
            # Add edge with BTC value as label
            net.add_edge(
                item["from_id"],
                item["to_id"],
                title=f"Amount: {item.get('btc_value')} BTC",
                label=f"{item.get('btc_value')} BTC",
                arrows="to",
                width=2
            )
    
    # Generate the HTML
    try:
        return net.show(output_path)
    except:
        # Fallback for older versions
        return net.generate_html(output_path)

def main(block_height=100):
    """Main function to fetch data and create visualization."""
    print("Fetching blockchain data...")
    response_data = fetch_bitcoin_block_data(block_height)
    graph_data = response_data['response']['result']['data']
    
    print("Creating visualization...")
    return create_transaction_graph(graph_data)

# For use in Jupyter notebook:
# Make sure to run this first:
# !pip install pyvis requests --upgrade

# Then run:
net = main(block_height=10000)
net  # Display the graph in the notebook

Fetching blockchain data...
Creating visualization...
bitcoin_transactions.html
